# Generate Consensus Signatures

**Gregory Way, 2019**

We do not have well-level information for the cell health data.
Therefore, we cannot map to cell painting replicates.

Instead, we generate consensus signatures for each treatment.
We generate consensus signatures in two ways.

1. Median consensus
2. MODZ (moderated z-score) transform used in the L1000 analysis paper ([Subramanian et al. 2017](https://doi.org/10.1016/j.cell.2017.10.049)).

We apply these transformations to both:

* Cell Painting Data
* Cell Health Assay Readout Data

In [1]:
import os
import numpy as np
import pandas as pd

from pycytominer.consensus import modz
from pycytominer import get_na_columns, aggregate

## Load Cell Painting Data

This will be our x matrix in machine learning appications.

In [2]:
batch = "CRISPR_PILOT_B1"
profile_dir = os.path.join("data", "profiles", batch)

all_profile_files = []
for plate in os.listdir(profile_dir):
    plate_dir = os.path.join(profile_dir, plate)
    
    if plate == '.DS_Store':
        continue

    for profile_file in os.listdir(plate_dir):
        if "feature_select" in profile_file:
            all_profile_files.append(os.path.join(plate_dir, profile_file))

In [3]:
all_profile_files

['data/profiles/CRISPR_PILOT_B1/SQ00014618/SQ00014618_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014611/SQ00014611_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014616/SQ00014616_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014617/SQ00014617_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014610/SQ00014610_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014615/SQ00014615_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014612/SQ00014612_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014613/SQ00014613_normalized_feature_select.csv.gz',
 'data/profiles/CRISPR_PILOT_B1/SQ00014614/SQ00014614_normalized_feature_select.csv.gz']

In [4]:
# Concatentate all cell painting datasets
x_df = (
    pd.concat(
        [pd.read_csv(x) for x in all_profile_files],
        sort=True
    )
    .rename(
        {
            "Image_Metadata_Plate": "Metadata_Plate",
            "Image_Metadata_Well": "Metadata_Well"
        },
        axis="columns")
    .drop(["Metadata_broad_sample"], axis="columns")
)

# Realign metadata column names
x_metadata_cols = x_df.columns[x_df.columns.str.startswith("Metadata")]
x_metadata_df = x_df.loc[:, x_metadata_cols]

x_df = x_df.drop(x_metadata_cols, axis="columns")
x_df = pd.concat([x_metadata_df, x_df], axis="columns")

print(x_df.shape)
x_df.head(2)

(3456, 1735)


,Metadata_Plate,Metadata_Well,Metadata_WellCol,Metadata_WellRow,Metadata_cell_line,Metadata_gene_name,Metadata_pert_name,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,SQ00014618,A01,1,A,HCC44,EMPTY,EMPTY,-0.958070,0.814507,-0.768176,...,-1.082920,-0.403869,-0.273727,-0.892703,-1.142401,-1.181333,-1.404747,1.008647,0.830481,1.025472
1,SQ00014618,A02,2,A,HCC44,MCL1,MCL1-5,0.547816,1.209206,-0.413024,...,0.746211,1.900085,1.554589,1.421829,0.556411,1.134919,0.348166,1.136893,1.948684,0.944716


## Load Cell Health Assay Data

This will be the y matrix in machine learning applications.

In [5]:
file = os.path.join("data", "labels", "normalized_cell_health_labels.tsv")
y_df = pd.read_csv(file, sep='\t').drop(["plate_name", "well_col", "well_row"], axis="columns")

print(y_df.shape)
y_df.head(2)

(2302, 72)


,cell_id,guide,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,0.655229,-0.565658,-0.839186,-0.513748,0.313600,0.263062,0.109983,-0.226513,...,0.281397,-0.279051,-0.429141,-0.177258,-0.920300,-0.139875,-0.016549,0.140570,NaN,NaN
1,ES2,AKT1-1,-0.251336,-0.816445,-0.525940,-0.819810,-0.450799,-0.811628,-0.468875,-0.167787,...,0.543716,-0.221588,-0.311041,-0.149198,-1.070176,-0.046783,0.268559,0.040163,-0.29248,0.008339


## Determine how many Cell Painting profiles have Cell Health status labels

In [6]:
x_groupby_cols = ["Metadata_gene_name", "Metadata_pert_name", "Metadata_cell_line"]

x_metacount_df = (
    x_df
    .loc[:, x_groupby_cols]
    .assign(n_measurements=1)
    .groupby(x_groupby_cols)
    .count()
    .reset_index()
    .assign(data_type="cell_painting")
    .merge(x_df.loc[:, x_groupby_cols + ["Metadata_Well", "Metadata_Plate"]],
           how="left",
           on=x_groupby_cols)
)

print(x_metacount_df.shape)
x_metacount_df.head(2)

(3456, 7)


,Metadata_gene_name,Metadata_pert_name,Metadata_cell_line,n_measurements,data_type,Metadata_Well,Metadata_Plate
0,AKT1,AKT1-1,A549,6,cell_painting,A03,SQ00014611
1,AKT1,AKT1-1,A549,6,cell_painting,O22,SQ00014611


In [7]:
y_groupby_cols = ["guide", "cell_id"]

y_metacount_df = (
    y_df
    .loc[:, y_groupby_cols]
    .assign(n_measurements=1)
    .groupby(y_groupby_cols)
    .count()
    .reset_index()
    .assign(data_type="cell_health")
)

print(y_metacount_df.shape)
y_metacount_df.head(2)

(364, 4)


,guide,cell_id,n_measurements,data_type
0,AKT1-1,A549,4,cell_health
1,AKT1-1,ES2,4,cell_health


In [8]:
all_measurements_df = (
    x_metacount_df
    .merge(
        y_metacount_df,
        left_on=["Metadata_pert_name", "Metadata_cell_line"],
        right_on=["guide", "cell_id"],
        suffixes=["_paint", "_health"],
        how="inner")
    .sort_values(by=["Metadata_cell_line", "Metadata_pert_name"])
    .reset_index(drop=True)
    .drop(["Metadata_Well", "guide", "cell_id"], axis="columns")
)

file = os.path.join("results", "all_profile_metadata.tsv")
all_measurements_df.to_csv(file, sep='\t', index=False)

print(all_measurements_df.shape)
all_measurements_df.head()

(3456, 8)


,Metadata_gene_name,Metadata_pert_name,Metadata_cell_line,n_measurements_paint,data_type_paint,Metadata_Plate,n_measurements_health,data_type_health
0,AKT1,AKT1-1,A549,6,cell_painting,SQ00014611,4,cell_health
1,AKT1,AKT1-1,A549,6,cell_painting,SQ00014611,4,cell_health
2,AKT1,AKT1-1,A549,6,cell_painting,SQ00014610,4,cell_health
3,AKT1,AKT1-1,A549,6,cell_painting,SQ00014610,4,cell_health
4,AKT1,AKT1-1,A549,6,cell_painting,SQ00014612,4,cell_health


## A. Apply Median Consensus Aggregation

### 1) To the Cell Painting Data

In [9]:
x_median_df = aggregate(
    x_df,
    strata=["Metadata_cell_line", "Metadata_pert_name"],
    features="infer",
    operation="median"
)


x_median_df = (
    x_median_df
    .query("Metadata_pert_name in @all_measurements_df.Metadata_pert_name.unique()")
    .query("Metadata_cell_line in @all_measurements_df.Metadata_cell_line.unique()")
    .reset_index(drop=True)
    .reset_index()
    .rename({"index": "Metadata_profile_id"}, axis='columns')
)
x_median_df.Metadata_profile_id = ["profile_{}".format(x) for x in x_median_df.Metadata_profile_id]

print(x_median_df.shape)
x_median_df.head()

(357, 1731)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,profile_0,A549,AKT1-1,0.128119,-0.523081,0.656915,0.340118,0.287256,0.0,-0.216196,...,0.943312,0.957411,0.980166,0.925170,0.694211,0.396416,0.943503,0.258312,0.146497,0.439103
1,profile_1,A549,AKT1-2,-0.151237,0.433182,0.560337,-0.279924,-0.018849,0.0,0.342880,...,0.606264,0.089609,-0.064143,0.230317,-0.062713,-0.133535,0.186883,0.225188,0.021131,0.002909
2,profile_2,A549,ARID1B-1,-0.050721,0.866254,-0.412500,1.169758,1.014053,0.0,-0.821827,...,-0.537697,0.043528,0.328203,-0.029953,-0.033669,0.071269,-0.317794,0.186336,0.207975,0.258786
3,profile_3,A549,ARID1B-2,0.500161,-0.464468,0.568008,0.353008,0.582756,0.0,-0.325497,...,0.087831,-0.230552,0.015408,-0.262526,0.010822,-0.025603,0.200284,0.725877,0.172991,0.567965
4,profile_4,A549,ATF4-1,3.247545,0.587303,-0.311460,-0.305051,0.083376,0.0,0.405565,...,0.225977,-0.167351,0.243971,-0.058260,-0.073743,-0.379313,0.288709,1.120616,2.423174,0.255682


In [10]:
# Output Profile Mapping for Downstream Analysis
profile_id_mapping_df = x_median_df.loc[:, x_median_df.columns.str.startswith("Metadata")]
file = os.path.join("data", "profile_id_metadata_mapping.tsv")
profile_id_mapping_df.to_csv(file, sep='\t', index=False)

profile_id_mapping_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


### 2) To the Cell Health Assay Data

In [11]:
cell_health_meta_features = ["cell_id", "guide"]
cell_health_features = y_df.drop(cell_health_meta_features, axis="columns").columns.tolist()
y_meta_merge_cols = ["Metadata_profile_id", "Metadata_pert_name", "Metadata_cell_line"]

In [12]:
y_median_df = aggregate(
    y_df,
    strata=cell_health_meta_features,
    features=cell_health_features,
    operation="median"
)

print(y_median_df.shape)
y_median_df.head()

(364, 72)


,cell_id,guide,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,A549,AKT1-1,0.008156,0.587977,0.018820,0.381501,0.176564,0.187675,-0.170616,0.039147,...,0.399842,0.000000,0.000000,0.000000,-0.118976,-0.132871,-0.121090,0.132882,0.806970,1.293984
1,A549,AKT1-2,0.056667,1.264627,0.241450,0.568443,0.235304,0.372684,-0.276888,-0.183445,...,0.101670,0.318027,0.132751,0.467027,0.621374,0.100032,0.074036,-0.099917,0.558041,1.151867
2,A549,ARID1B-1,0.111163,1.092964,0.151393,0.290203,0.402121,0.481700,-0.276980,-0.149979,...,0.080701,0.339100,0.165161,0.247058,0.598093,0.055951,0.042014,-0.055920,-0.393937,0.103202
3,A549,ARID1B-2,-0.061528,0.320829,-0.091007,0.141819,-0.378769,-0.288693,-0.108741,-0.300783,...,0.265754,0.098699,0.138654,0.000000,0.371930,-0.063935,-0.055160,0.063946,0.210005,0.055291
4,A549,ATF4-1,3.967818,0.003400,3.268615,-2.246887,2.891737,2.878938,2.853995,1.243444,...,-2.343919,0.000000,0.000000,0.000000,-0.089544,0.141535,0.131393,-0.141397,-0.631390,0.106477


In [13]:
y_median_df = (
    y_median_df
    .reset_index(drop=True)
    .merge(
        x_median_df.loc[:, y_meta_merge_cols],
        left_on=["guide", "cell_id"],
        right_on=["Metadata_pert_name", "Metadata_cell_line"],
        how="right"
    )
)

# Get columns in correct order
y_columns = (
    y_meta_merge_cols +
    y_median_df
    .loc[:, ~y_median_df.columns.str.startswith("Metadata_")]
    .columns
    .tolist()
)

y_median_df = (
    y_median_df
    .loc[:, y_columns]
    .drop(["guide", "cell_id"], axis="columns")
)

print(y_median_df.shape)
y_median_df.head(5)

(357, 73)


,Metadata_profile_id,Metadata_pert_name,Metadata_cell_line,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,AKT1-1,A549,0.008156,0.587977,0.018820,0.381501,0.176564,0.187675,-0.170616,...,0.399842,0.000000,0.000000,0.000000,-0.118976,-0.132871,-0.121090,0.132882,0.806970,1.293984
1,profile_1,AKT1-2,A549,0.056667,1.264627,0.241450,0.568443,0.235304,0.372684,-0.276888,...,0.101670,0.318027,0.132751,0.467027,0.621374,0.100032,0.074036,-0.099917,0.558041,1.151867
2,profile_2,ARID1B-1,A549,0.111163,1.092964,0.151393,0.290203,0.402121,0.481700,-0.276980,...,0.080701,0.339100,0.165161,0.247058,0.598093,0.055951,0.042014,-0.055920,-0.393937,0.103202
3,profile_3,ARID1B-2,A549,-0.061528,0.320829,-0.091007,0.141819,-0.378769,-0.288693,-0.108741,...,0.265754,0.098699,0.138654,0.000000,0.371930,-0.063935,-0.055160,0.063946,0.210005,0.055291
4,profile_4,ATF4-1,A549,3.967818,0.003400,3.268615,-2.246887,2.891737,2.878938,2.853995,...,-2.343919,0.000000,0.000000,0.000000,-0.089544,0.141535,0.131393,-0.141397,-0.631390,0.106477


In [14]:
# Confirm that matrices are aligned
pd.testing.assert_series_equal(x_median_df.Metadata_profile_id,
                               y_median_df.Metadata_profile_id, check_names=True)

# Are the guides aligned?
pd.testing.assert_series_equal(x_median_df.Metadata_pert_name,
                               y_median_df.Metadata_pert_name, check_names=True)

# Are the cells aligned?
pd.testing.assert_series_equal(x_median_df.Metadata_cell_line,
                               y_median_df.Metadata_cell_line, check_names=True)

## B. Apply the MODZ Consensus Aggregation

### 1) To the Cell Painting Data

In [15]:
x_consensus_df = modz(
    x_df,
    replicate_columns=["Metadata_cell_line", "Metadata_pert_name"],
    precision=5
)

x_consensus_df.head()

Cells_AreaShape_Area  \
Metadata_cell_line Metadata_pert_name                         
A549               AKT1-1                          0.082039   
                   AKT1-2                         -0.127207   
                   ARID1B-1                        0.160263   
                   ARID1B-2                        0.489093   
                   ATF4-1                          3.416773   

                                       Cells_AreaShape_Center_X  \
Metadata_cell_line Metadata_pert_name                             
A549               AKT1-1                             -0.005152   
                   AKT1-2                              0.393959   
                   ARID1B-1                            0.706940   
                   ARID1B-2                           -0.007086   
                   ATF4-1                             -0.120053   

                                       Cells_AreaShape_Center_Y  \
Metadata_cell_line Metadata_pert_name                             
A549               AKT1-1                              0.007516   
                   AKT1-2                              0.456300   
                   ARID1B-1                           -0.351952   
                   ARID1B-2                            0.368221   
                   ATF4-1                             -0.075411   

                                       Cells_AreaShape_Compactness  \
Metadata_cell_line Metadata_pert_name                                
A549               AKT1-1                                -0.262496   
                   AKT1-2                                -0.193656   
                   ARID1B-1                               0.821411   
                   ARID1B-2                               0.430873   
                   ATF4-1                                -0.082592   

                                       Cells_AreaShape_Eccentricity  \
Metadata_cell_line Metadata_pert_name                                 
A549               AKT1-1                                 -0.096337   
                   AKT1-2                                 -0.020202   
                   ARID1B-1                                0.657121   
                   ARID1B-2                                0.707579   
                   ATF4-1                                  0.454871   

                                       Cells_AreaShape_EulerNumber  \
Metadata_cell_line Metadata_pert_name                                
A549               AKT1-1                                      0.0   
                   AKT1-2                                      0.0   
                   ARID1B-1                                    0.0   
                   ARID1B-2                                    0.0   
                   ATF4-1                                      0.0   

                                       Cells_AreaShape_Extent  \
Metadata_cell_line Metadata_pert_name                           
A549               AKT1-1                            0.111426   
                   AKT1-2                            0.273250   
                   ARID1B-1                         -0.506445   
                   ARID1B-2                         -0.367911   
                   ATF4-1                            0.462898   

                                       Cells_AreaShape_FormFactor  \
Metadata_cell_line Metadata_pert_name                               
A549               AKT1-1                               -0.195349   
                   AKT1-2                                0.357932   
                   ARID1B-1                              0.324896   
                   ARID1B-2                              0.094606   
                   ATF4-1                               -1.637233   

                                       Cells_AreaShape_MajorAxisLength  \
Metadata_cell_line Metadata_pert_name                                    
A549               AKT1-1                                     0.122201   
               

In [16]:
x_consensus_df = (
    x_consensus_df
    .reset_index()
    .query("Metadata_pert_name in @all_measurements_df.Metadata_pert_name.unique()")
    .query("Metadata_cell_line in @all_measurements_df.Metadata_cell_line.unique()")
    .reset_index(drop=True)
    .reset_index()
    .rename({"index": "Metadata_profile_id"}, axis='columns')
)
x_consensus_df.Metadata_profile_id = ["profile_{}".format(x) for x in x_consensus_df.Metadata_profile_id]

print(x_consensus_df.shape)
x_consensus_df.head(5)

(357, 1731)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,profile_0,A549,AKT1-1,0.082039,-0.005152,0.007516,-0.262496,-0.096337,0.0,0.111426,...,0.911852,1.227693,1.103665,1.200197,0.596277,0.537276,0.552541,0.354905,0.183667,0.538379
1,profile_1,A549,AKT1-2,-0.127207,0.393959,0.456300,-0.193656,-0.020202,0.0,0.273250,...,0.526762,0.365948,0.192068,0.406909,0.006834,0.051908,0.087319,0.156875,0.146667,0.082659
2,profile_2,A549,ARID1B-1,0.160263,0.706940,-0.351952,0.821411,0.657121,0.0,-0.506445,...,-0.536363,-0.028874,0.112922,-0.104128,-0.048906,0.072595,-0.198982,0.325876,0.186032,0.342683
3,profile_3,A549,ARID1B-2,0.489093,-0.007086,0.368221,0.430873,0.707579,0.0,-0.367911,...,0.006326,-0.334649,-0.269844,-0.255930,-0.107642,-0.099935,0.093797,0.606730,0.427497,0.672805
4,profile_4,A549,ATF4-1,3.416773,-0.120053,-0.075411,-0.082592,0.454871,0.0,0.462898,...,0.410653,-0.249195,0.061380,-0.142925,0.008937,-0.204777,0.216734,1.356973,2.361216,0.344510


### 2) To the Cell Health Assay Data

In [17]:
y_consensus_df = modz(
    y_df,
    features=cell_health_features,
    replicate_columns=cell_health_meta_features,
    precision=5)

print(y_consensus_df.shape)
y_consensus_df.head()

(364, 70)


cc_all_high_n_spots_h2ax_mean  \
cell_id guide                                     
A549    AKT1-1                        -0.005795   
        AKT1-2                         0.050169   
        ARID1B-1                       0.118598   
        ARID1B-2                      -0.072919   
        ATF4-1                         4.286179   

                  cc_all_large_notround_polynuclear_mean  \
cell_id guide                                              
A549    AKT1-1                                  0.580351   
        AKT1-2                                  1.277730   
        ARID1B-1                                1.198685   
        ARID1B-2                                0.317079   
        ATF4-1                                  0.007467   

                  cc_all_large_round_polyploid_mean  cc_all_n_objects  \
cell_id guide                                                           
A549    AKT1-1                             0.013975          0.381958   
        AKT1-2                             0.241808          0.577422   
        ARID1B-1                           0.165140          0.330071   
        ARID1B-2                          -0.089281          0.155305   
        ATF4-1                             3.284383         -2.246762   

                  cc_all_n_spots_mean  cc_all_n_spots_per_nucleus_area_mean  \
cell_id guide                                                                 
A549    AKT1-1               0.150696                              0.162511   
        AKT1-2               0.220829                              0.366989   
        ARID1B-1             0.417723                              0.514065   
        ARID1B-2            -0.385316                             -0.300279   
        ATF4-1               3.015881                              3.065773   

                  cc_all_nucleus_area_mean  cc_all_nucleus_roundness_mean  \
cell_id guide                                                               
A549    AKT1-1                   -0.167603                       0.040322   
        AKT1-2                   -0.278044                      -0.182571   
        ARID1B-1                 -0.308749                      -0.172735   
        ARID1B-2                 -0.120261                      -0.299718   
        ATF4-1                    2.854124                       1.246508   

                  cc_cc_edu_pos_mean  cc_cc_g1_mean  ...  vb_num_live_cells  \
cell_id guide                                        ...                      
A549    AKT1-1              0.091643       0.043915  ...           0.438339   
        AKT1-2              0.893453      -1.023968  ...           0.067568   
        ARID1B-1            0.455015      -0.606599  ...           0.100365   
        ARID1B-2            0.131195      -0.221492  ...           0.283802   
        ATF4-1             -1.623145       0.923187  ...          -2.434136   

                  vb_percent_all_apoptosis  vb_percent_all_early_apoptosis  \
cell_id guide                                                                
A549    AKT1-1                    0.059414                        0.082424   
        AKT1-2                    0.256141                        0.132834   
        ARID1B-1                  0.299229                        0.249557   
        ARID1B-2                  0.143096                        0.136790   
        ATF4-1                    0.315381                        0.332291   

                  vb_percent_all_late_apoptosis  vb_percent_caspase_dead_only  \
cell_id guide                                                                   
A549    AKT1-1                         0.000000                     -0.065050   
        AKT1-2                         0.386327                      0.575026   
        ARID1B-1                       0.283246                      0.501941   
        ARID1B-2                       0.111855                      0.493883   
        ATF4-1                         0.19287

In [18]:
y_consensus_df = (
    y_consensus_df
    .reset_index()
    .reset_index(drop=True)
    .merge(
        x_consensus_df.loc[:, y_meta_merge_cols],
        left_on=["guide", "cell_id"],
        right_on=["Metadata_pert_name", "Metadata_cell_line"],
        how="right"
    )
    .loc[:, y_columns]
    .drop(["guide", "cell_id"], axis="columns")
)

print(y_consensus_df.shape)
y_consensus_df.head(5)

(357, 73)


,Metadata_profile_id,Metadata_pert_name,Metadata_cell_line,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,AKT1-1,A549,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,...,0.438339,0.059414,0.082424,0.000000,-0.065050,-0.020236,-0.007970,0.020263,0.408214,0.654575
1,profile_1,AKT1-2,A549,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,...,0.067568,0.256141,0.132834,0.386327,0.575026,0.225091,0.220461,-0.224965,0.284962,0.567898
2,profile_2,ARID1B-1,A549,0.118598,1.198685,0.165140,0.330071,0.417723,0.514065,-0.308749,...,0.100365,0.299229,0.249557,0.283246,0.501941,0.055517,0.047697,-0.055445,-0.363766,-0.205937
3,profile_3,ARID1B-2,A549,-0.072919,0.317079,-0.089281,0.155305,-0.385316,-0.300279,-0.120261,...,0.283802,0.143096,0.136790,0.111855,0.493883,-0.084415,-0.085658,0.084439,0.198285,-0.162976
4,profile_4,ATF4-1,A549,4.286179,0.007467,3.284383,-2.246762,3.015881,3.065773,2.854124,...,-2.434136,0.315381,0.332291,0.192878,0.509217,0.100668,0.104857,-0.100596,-0.811271,-0.092505


In [19]:
# Confirm that matrices are aligned
pd.testing.assert_series_equal(x_consensus_df.Metadata_profile_id,
                               y_consensus_df.Metadata_profile_id, check_names=True)

# Are the guides aligned?
pd.testing.assert_series_equal(x_consensus_df.Metadata_pert_name,
                               y_consensus_df.Metadata_pert_name, check_names=True)

# Are the cells aligned?
pd.testing.assert_series_equal(x_consensus_df.Metadata_cell_line,
                               y_consensus_df.Metadata_cell_line, check_names=True)

## Output Consensus Signatures

In [20]:
file = os.path.join("data", "consensus", "cell_painting_median.tsv.gz")
x_median_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "consensus", "cell_health_median.tsv.gz")
y_median_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "consensus", "cell_painting_modz.tsv.gz")
x_consensus_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "consensus", "cell_health_modz.tsv.gz")
y_consensus_df.to_csv(file, sep="\t", index=False)